#### MODEL TUNING CODE
##### Author : Kunal Kalwankar

In [ ]:
#Imports
import numpy as np
import pandas as pd
import os      
import torch
import json
!pip install simplet5
import torch
import json 
from sklearn.model_selection import train_test_split

In [ ]:
# Import pretrained model
from simplet5 import SimpleT5
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

In [ ]:
#importing pre deicded testids for all the topics. These were split 90-10 only once for all iterations and saved
import os
os.listdir('/kaggle/input/cnndm-tagged-subset')
data = pd.read_csv('/kaggle/input/cnndm-tagged-subset/masterdata.csv')
t1 = pd.read_csv('/kaggle/input/testids/testids_1_to_5.csv')
t2 = pd.read_csv('/kaggle/input/testids/testids_2_to_5.csv')
t3 = pd.read_csv('/kaggle/input/testids/testids_3_to_5.csv')
t4 = pd.read_csv('/kaggle/input/testids/testids_4_to_5.csv')
t5 = pd.read_csv('/kaggle/input/testids/testids_5_to_5.csv')
t = pd.concat([t1,t2,t3,t4,t5])
test = data[data['id'].isin(list(t['TESTIDS']))]
train = data[~data['id'].isin(list(t['TESTIDS']))]

In [ ]:
# Hyperparameters
MAX_LEN = 512
SUMMARY_LEN = 150

# Getting test datapoints
test_article = list(test['article'])[250:]
test_highlight = list(test['highlights'])[250:]

In [ ]:
#import pretrained model
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
device = torch.device('cpu')

In [ ]:
#preprocess the test inputs sequenctially to get in the final format
preprocess_text = [i.strip().replace("\n","") for i in test_article]
t5_prepared_Text = ["summarize: "+i for i in preprocess_text]
tokenized_text = [tokenizer.encode(i, return_tensors="pt").to(device) for i in t5_prepared_Text]
summary_ids = [no_tune_model.generate(i,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=SUMMARY_LEN,
                                    early_stopping=True) for i in tokenized_text]

output = [tokenizer.decode(i[0], skip_special_tokens=True) for i in summary_ids]


#Save the predicted outcode for metric calculator
notune = pd.DataFrame({'outcome':list(output)})
notune.to_csv('outcomenotune_t5base_1.csv')